In [3]:
import torch
import torch.nn as nn
import torchvision.models as models

# Load the pre-trained models
resnet50 = models.resnet50(pretrained=True)
resnet101 = models.resnet101(pretrained=True)
vgg16 = models.vgg16(pretrained=True)

# Freeze the parameters of the pre-trained models
for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet101.parameters():
    param.requires_grad = False

for param in vgg16.parameters():
    param.requires_grad = False

# Extract the features from the pre-trained models
resnet50_conv = nn.Sequential(*list(resnet50.children())[:-1])
resnet101_conv = nn.Sequential(*list(resnet101.children())[:-1])
vgg16_conv = nn.Sequential(*list(vgg16.features.children())[:-1])

# Add the batch normalization layer and flattened layer
batch_norm = nn.BatchNorm1d(1000)

resnet50_model = nn.Sequential(resnet50_conv, nn.Flatten(), batch_norm)
resnet101_model = nn.Sequential(resnet101_conv, nn.Flatten(), batch_norm)
vgg16_model = nn.Sequential(vgg16_conv, nn.Flatten(), batch_norm)

# Print the models
print(resnet50_model)
print(resnet101_model)
print(vgg16_model)


/home/weber50432/miniconda3/envs/ML/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/weber50432/miniconda3/envs/ML/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/weber50432/miniconda3/envs/ML/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can al

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the neural network
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(25088, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Define the dataset
class MyDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y

# Define the training function
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Define the evaluation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    return running_loss / len(dataloader)

# Load the data
data = np.load('data.npy')
targets = np.load('targets.npy')

# Split the data into train and validation sets
split_idx = int(len(data) * 0.5)
train_data = data[:split_idx]
train_targets = targets[:split_idx]
val_data = data[split_idx:]
val_targets = targets[split_idx:]

# Create the dataloaders
train_dataset = MyDataset(train_data, train_targets)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = MyDataset(val_data, val_targets)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the model and optimizer
model = MyNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.BCELoss()

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model for 50 iterations
best_val_loss = np.inf
for epoch in range(50):
    train_loss = train(model,train_dataloader, criterion, optimizer, device)
val_loss = evaluate(model, val_dataloader, criterion, device)
print('Epoch {} Train Loss: {:.4f} Val Loss: {:.4f}'.format(epoch+1, train_loss, val_loss))
# Save the best model
if val_loss < best_val_loss:
    best_val_loss = val_loss
    torch.save(model.state_dict(), 'best_model.pt')



FileNotFoundError: [Errno 2] No such file or directory: 'data.npy'